In [1]:
# Importing Libraries
import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS 
import numpy as np
from matplotlib import pyplot as plt

In [2]:
#Loading the data
df = pd.read_csv("Data/Processed/citibike_2022_with_weather.csv")

/var/folders/wf/s0crnzgd66b7pdj9_7_b6mpw0000gn/T/ipykernel_18459/2851044232.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Data/Processed/citibike_2022_with_weather.csv")


# Aggregrating Data

In [3]:
df['value'] = 1

In [4]:
#Taking a 30% RANDOM sample
df_sample = df.sample(frac=0.3, random_state=42)
df_sample.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,_merge,value
15115901,F66F3FD176B86390,electric_bike,2022-08-03 09:32:03.967,2022-08-03 10:32:12.464,Delancey St & Eldridge St,5414.07,NaN,NaN,40.719440,-73.991804,40.670000,-73.990000,casual,2022-08-03,27.6,both,1
753105,9FA92A23DAA636C4,electric_bike,2022-09-16 08:02:48.369,2022-09-16 08:06:25.251,1 Ave & E 78 St,7020.09,E 88 St & 1 Ave,7235.13,40.771404,-73.953517,40.778301,-73.948813,member,2022-09-16,19.6,both,1
23821092,021C712DBEF40F84,classic_bike,2022-10-12 17:31:51.550,2022-10-12 17:46:15.185,St James Pl & Oliver St,5238.05,E 1 St & Bowery,5636.13,40.713079,-73.998512,40.724861,-73.992131,casual,2022-10-12,17.9,both,1
17525642,B23EC0300455C2AD,electric_bike,2022-08-14 15:53:17.790,2022-08-14 16:16:22.126,E 4 St & 2 Ave,5593.04,Columbia Heights & Cranberry St,4829.01,40.726239,-73.989637,40.700379,-73.995481,casual,2022-08-14,24.0,both,1
6397983,7E2CB79CE92BDB24,classic_bike,2022-05-10 21:09:56.750,2022-05-10 21:17:36.103,E 25 St & 2 Ave,6046.02,E 16 St & 5 Ave,6022.04,40.739126,-73.979738,40.737262,-73.992390,member,2022-05-10,15.4,both,1


In [5]:
#grouping
df_group = (
    df_sample.groupby(
        [
            'start_station_name',
            'end_station_name',
            'start_lat',
            'start_lng',
            'end_lat',
            'end_lng'
        ],
        as_index=False
    )['value'].count())

In [6]:
#Renaming the columns
df_group.rename(columns={
    'start_station_name': 'start_station',
    'end_station_name': 'end_station',
    'value': 'trips'
}, inplace=True)

df_map = df_group.copy()
df_map.head()

,start_station,end_station,start_lat,start_lng,end_lat,end_lng,trips
0,1 Ave & E 110 St,1 Ave & E 110 St,40.792199,-73.937879,40.792327,-73.9383,1
1,1 Ave & E 110 St,1 Ave & E 110 St,40.792230,-73.937900,40.792327,-73.9383,1
2,1 Ave & E 110 St,1 Ave & E 110 St,40.792243,-73.937957,40.792327,-73.9383,1
3,1 Ave & E 110 St,1 Ave & E 110 St,40.792257,-73.937966,40.792327,-73.9383,1
4,1 Ave & E 110 St,1 Ave & E 110 St,40.792257,-73.937915,40.792327,-73.9383,1


In [7]:
#Validating counts
df_map['trips'].sum(), df_sample.shape

(np.int64(8930454), (8951642, 17))

# Initalizing Kepler.gl Map

In [9]:
m = KeplerGl(height=700, data={"NYC Trips (30% Sample)": df_map})
m

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'NYC Trips (30% Sample)':                  start_station            end_station  start_lat  sta…

In [14]:
#Saving Map
config = m.config

m.save_to_html(
    file_name="CitiBike_Trips_Kepler_30percent.html",
    read_only=False,
    config=config)

Map saved to CitiBike_Trips_Kepler_30percent.html!


In [13]:
import json
with open("kepler_config.json", "w") as f:
    json.dump(config, f)